# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from maddpg import MADDPG
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
def ddpg(maddpg, n_episodes=3000, print_every=100, train_every=20, repeat_training=10, save_every=500):
    """ Deep Deterministic Policy Gradients (DDPG) for multiple agents.
        
        Params
        ======
        maddpg (MADDPG): an object encapsulating multiple agents to be trained
        n_episodes (int): maximum number of episodes
        print_every (int): print training stats every 'print_every' episodes
        train_every (int): train agents every 'train_every' episodes
        repeat_training (int): number of times all agents are trained every episode training is invoked
        save_every (int): save checkpoint every 'save_every' episodes

        Returns
        ======
        scores (list(float)): a list containing the mean score (averaged over all agents) for each episode

    """

    total_scores = []

    for i_episode in range(1,n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        maddpg.reset()
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                           # initialize the score (for each agent)

        t = 1
        while True:
                                                                # select an action (for each agent)
            actions = maddpg.act(states, add_noise=True)

            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            
            # add experiences of both agents to a shared replay buffer
            maddpg.add_to_buffer(states, actions, rewards, next_states, dones)

            # learn
            if t%train_every==0:
                for _ in range(repeat_training):
                    maddpg.train()

            scores += rewards                                  # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            t += 1
            
            if np.any(dones):                                  # exit loop if episode finished
                break

        total_scores.append(np.mean(scores))


        # print training stats
        if i_episode%print_every==0:
            print('\nEpisode {:d}:\tAverage Score: {:.2f}\tScore: {:.2f}\tMin: {:.2f}\tMax: {:.2f}'.format(i_episode,np.mean(total_scores[-100:]),  np.mean(scores), np.min(total_scores[-print_every:]), np.max(total_scores[-print_every:])))

        # save networks
        if i_episode%save_every==0:
            maddpg.save(prefix='_episode_{:d}'.format(i_episode))
            print('\nEpisode: {:d}: Networks saved'.format(i_episode))

        # training finished when target performance reached
        if i_episode >= 100 and np.mean(total_scores[-100:]) >= 0.5:
            print('\nFinished! Environment solved in {:d} episodes with an average score of {:.2f} over the last 100 episodes!'.format())
            break
    
    print('\nSaving networks...')
    maddpg.save('_final')
    print('\nDone!')

    return total_scores


In [ ]:

maddpg = MADDPG(state_size=state_size, action_size=action_size, num_agents=num_agents, random_seed=7, buffer_size=int(1e6), batch_size=256)

In [ ]:
scores = ddpg(maddpg, n_episodes=5000, print_every=100, train_every=4, repeat_training=3, save_every=500)

### 4. Results

#### 4.1 Plot

In [ ]:
# Compute rolling average
average_scores = [np.mean(scores[i-100:i]) for i in range(len(scores))]

# Plot 
fig = plt.figure()
plt.plot(scores, color='b', label='Score')
plt.plot(average_scores, color='r', label='Average Score')
plt.plot(np.ones(len(scores))*0.5, linestyle='dashed', color='y', label='Target')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend()

plt.show()

#### 4.2 Interact with the environment

In [ ]:
# restore saved checkpoints
maddpg.load(agent_0_actor='checkpoint_agent_0_actor_final.pth', \
            agent_0_critic='checkpoint_critic_0_critic_final.pth', \
            agent_1_actor='checkpoint_agent_1_actor_final.pth', \
            agent_1_critic='checkpoint_critic_1_critic_final.pth')

In [ ]:
# Use trained agents and interact with the environment
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:        
        actions = maddpg.act(states, add_noise=False)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

In [ ]:
env.close()